## ACGAN ##

In [1]:
import torch
import torchvision
from torchvision import utils
from torch.utils.data import DataLoader
from torch import nn
from torch.autograd import Variable
from pytorch_gan_metrics import get_inception_score
from tqdm import tqdm
import os
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cuda


In [2]:
def create_CIFAR10_dataloaders(batch_size): 
    transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(), 
        torchvision.transforms.Resize(32), 
        torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    train_CIFAR10_set = torchvision.datasets.CIFAR10(root='./cifar10/', train=True, download=True, transform=transform)
    test_CIFAR10_set = torchvision.datasets.CIFAR10(root='./cifar10/', train=False, download=True, transform=transform)

    train_CIFAR10_dataloader = DataLoader(train_CIFAR10_set, batch_size=batch_size, shuffle=True, drop_last=True)
    test_CIFAR10_dataloader = DataLoader(test_CIFAR10_set, batch_size=batch_size, shuffle=True, drop_last=True)
    return train_CIFAR10_dataloader, test_CIFAR10_dataloader

print("Downloading CIFAR10 dataset...")
batch_size = 64
train_dataloader, test_dataloader = create_CIFAR10_dataloaders(batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class Generator_ACGAN(nn.Module):
    def __init__(self):
        super(Generator_ACGAN, self).__init__()
        self.emb = nn.Embedding(10, 100)
        self.fc = nn.Linear(100, 128 * 8 ** 2)
        self.main = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 3, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, noise, labels):
        x = torch.mul(self.emb(labels), noise)
        x = self.fc(x)
        x = x.view(x.shape[0], 128, 8, 8)
        x = self.main(x)
        return x

class Discriminator_ACGAN(nn.Module):
    def __init__(self):
        super(Discriminator_ACGAN, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.main = nn.Sequential(
            *discriminator_block(3, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        self.adv_layer = nn.Sequential(nn.Linear(128 * 2 ** 2, 1), nn.Sigmoid())
        self.aux_layer = nn.Sequential(nn.Linear(128 * 2 ** 2, 10), nn.Softmax())

    def forward(self, img):
        x = self.main(img)
        x = x.view(x.shape[0], -1)
        validity = self.adv_layer(x)
        label = self.aux_layer(x)
        return validity, label


print("Instantiating DCGAN generator and discriminator...")
acgan_generator = Generator_ACGAN()
acgan_discriminator = Discriminator_ACGAN()
acgan_generator.to(device)
acgan_discriminator.to(device)
print()

Instantiating DCGAN generator and discriminator...



In [4]:
epochs=50
learning_rate = 2e-4

def train(generator, discriminator, train_dataloader):
    source_criterion = nn.BCELoss()
    class_criterion = nn.NLLLoss()
    optim_generator = torch.optim.Adam(generator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
    optim_discriminator = torch.optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(0.5, 0.999))

    if not os.path.exists('train_generated_images_acgan_real/'): 
        os.makedirs('train_generated_images_acgan_real')
    if not os.path.exists('train_generated_images_acgan_fake/'): 
        os.makedirs('train_generated_images_acgan_fake')
        
    inception_score_file = open("inception_score_acgan.csv", "w")
    inception_score_file.write('epoch, inception_score \n')

    for epoch in tqdm(range(epochs)): 
        for images, labels in train_dataloader:
            batch_size = images.shape[0]
            real_images = Variable(images.type(torch.cuda.FloatTensor)).to(device)
            real_labels = Variable(labels.type(torch.cuda.LongTensor)).to(device)

            # adversarial ground truth
            fake = torch.zeros(batch_size).to(device)
            valid = torch.ones(batch_size).to(device)

            ### train generator
            optim_generator.zero_grad()
            z = Variable(torch.cuda.FloatTensor(np.random.normal(0, 1, (batch_size, 100))))
            generated_labels = Variable(torch.cuda.LongTensor(np.random.randint(0, 10, batch_size)))

            # generate image batch
            generated_images = generator(z, generated_labels)

            # compute generator loss, optimize generator
            validity, predicted_label = discriminator(generated_images)
            gen_loss = 0.5 * (source_criterion(validity, valid.unsqueeze(1)) + class_criterion(predicted_label, generated_labels))
            gen_loss.backward()
            optim_generator.step()

            ### train discriminator
            optim_discriminator.zero_grad()

            # compute real images loss
            real_pred, real_aux = discriminator(real_images)
            disc_loss_real = 0.5 * (source_criterion(real_pred, valid.unsqueeze(1)) + class_criterion(real_aux, real_labels))

            # compute fake images loss
            fake_pred, fake_aux = discriminator(generated_images.detach())
            disc_loss_fake = 0.5 * (source_criterion(fake_pred, fake.unsqueeze(1)) + class_criterion(fake_aux, generated_labels))

            # compute overall discriminator loss, optimize discriminator
            disc_loss = 0.5 * (disc_loss_real + disc_loss_fake)
            disc_loss.backward()
            optim_discriminator.step()

        # compute inception score and samples every epoch
        z = Variable(torch.cuda.FloatTensor(np.random.normal(0, 1, (batch_size, 100))))
        generated_labels = Variable(torch.cuda.LongTensor(np.random.randint(0, 10, batch_size)))
        samples = generator(z, generated_labels)

        # normalize to [0, 1]
        samples = samples.mul(0.5).add(0.5)
        
        assert 0 <= samples.min() and samples.max() <= 1
        inception_score, inception_score_std = get_inception_score(samples)
        print("epoch: " + str(epoch) + ', inception score: ' + str(round(inception_score, 2)) + ' ± ' + str(round(inception_score_std, 2)))

        samples = samples[:64].data.cpu()
        utils.save_image(samples, 'train_generated_images_acgan_fake/epoch_{}.png'.format(str(epoch)))
        utils.save_image(real_images, 'train_generated_images_acgan_real/epoch_{}.png'.format(str(epoch)))
        
        inception_score_file.write(str(epoch) + ', ' + str(round(inception_score, 2)) + '\n')

    inception_score_file.close()

In [5]:
# train ACGAN
print("training ACGAN model...")
train(acgan_generator, acgan_discriminator, train_dataloader)

# save ACGAN to file
#print("saving ACGAN model to file...")
#torch.save(acgan_generator.state_dict(), 'acgan_generator.pkl')
#torch.save(acgan_discriminator.state_dict(), 'acgan_discriminator.pkl')

training ACGAN model...


  0%|          | 0/50 [00:00<?, ?it/s]c:\Users\FJ\.conda\envs\cpsc-8430\lib\site-packages\torch\nn\modules\container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
  2%|▏         | 1/50 [01:06<53:56, 66.04s/it]

epoch: 0, inception score: 1.59 ± 0.2


  4%|▍         | 2/50 [01:38<37:07, 46.40s/it]

epoch: 1, inception score: 1.44 ± 0.13


...

epoch: 47, inception score: 1.84 ± 0.28


 98%|█████████▊| 49/50 [20:06<00:21, 21.72s/it]

epoch: 48, inception score: 1.73 ± 0.15


100%|██████████| 50/50 [20:37<00:00, 24.75s/it]

epoch: 49, inception score: 2.04 ± 0.26
saving ACGAN model to file...


In [6]:
def generate_images(generator):
    z = Variable(torch.cuda.FloatTensor(np.random.normal(0, 1, (batch_size, 100))))
    generated_labels = Variable(torch.cuda.LongTensor(np.random.randint(0, 10, batch_size)))
    samples = generator(z, generated_labels)
    
    samples = samples.mul(0.5).add(0.5)
    samples = samples.data.cpu()
    grid = utils.make_grid(samples)
    print("Grid of 8x8 images saved to 'acgan_generated_images.png'.")
    utils.save_image(grid, 'acgan_generated_images.png')

def load_model(model, model_filename): 
    model.load_state_dict(torch.load(model_filename))

# load trained model and generate sample images
print("loading ACGAN model...")
load_model(acgan_generator, 'acgan_generator.pkl')
load_model(acgan_discriminator, 'acgan_discriminator.pkl')

generate_images(acgan_generator)

loading ACGAN model...
Grid of 8x8 images saved to 'acgan_generated_images.png'.
